In [ ]:
print(inv_label_map)import sys
import os
import subprocess

# Make sure you have all of these packages installed, e.g. via pip
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy
from skimage import io
from scipy import ndimage
from IPython.display import display

import cv2

import tifffile as tiff

from PIL import Image, ImageFilter, ImageEnhance, ImageOps

from keras.models import Sequential
from keras.models import Model
from keras.layers import Input, Flatten, Dense, merge, Conv2D, Convolution2D, MaxPooling2D, UpSampling2D, Reshape, core, Dropout, ZeroPadding2D, Activation
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K

%matplotlib inline


PLANET_KAGGLE_ROOT = os.path.abspath("../input/")
#PLANET_KAGGLE_JPEG_DIR = os.path.join(PLANET_KAGGLE_ROOT, 'train-jpg')
PLANET_KAGGLE_LABEL_CSV = os.path.join(PLANET_KAGGLE_ROOT, 'train.csv')
train_tiff_dir = os.path.join(PLANET_KAGGLE_ROOT, 'train-tif/')

In [ ]:
df_train = pd.read_csv(PLANET_KAGGLE_LABEL_CSV)
imagenes = len(df_train)
im_train = 100
im_test = 80

flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

print(inv_label_map)

In [ ]:
imagenes = len(df_train)
porcentaje = 0.1
im_train = int(imagenes * porcentaje)
im_test = int(im_train * (1- porcentaje))

print(im_train)

In [ ]:
x_train = []
x_test = []
y_train = []

df_train = pd.read_csv(PLANET_KAGGLE_LABEL_CSV)
imagenes = len(df_train)

porcentaje = 0.1
im_train = int(imagenes * porcentaje)
im_test = int(im_train * (1- porcentaje))

formato = 'tif'
baseimage = formato
codimage = '{}.'+ baseimage

for f, tags in df_train.values[:im_train]:
   
    img = tiff.imread(train_tiff_dir + '{}.tif'.format(f))
    #img = np.rollaxis(img, 0, 3)
    img = cv2.resize(img, (32, 32)) 

    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 

    x_train.append(img)
    y_train.append(targets)
    
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float16) / 255.
x_train = x_train.transpose((0,3,1,2))

print(x_train.shape)
print(y_train.shape)

split = im_test
x_train, x_valid, y_train, y_valid = x_train[:split], x_train[split:], y_train[:split], y_train[split:]

im_size = x_train.shape[1:4]
bands, h, w = im_size

print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)

print(y_valid.shape)

In [ ]:
from __future__ import division
import operator
smooth = 17
def personal_coef(y_true, y_pred):
    # __author__ = Vladimir Iglovikov
    clases_true = K.sum(y_true)
    clases_pred = K.sum(y_pred)

    val = abs(clases_true - clases_pred)
    net = K.dot(val, 1/17)
    
    resultado = 1 - net

    return resultado



inputs = Input((bands, w, h))
    
x = (Conv2D(32, (3, 3), activation='relu'))(inputs)
x = (MaxPooling2D(pool_size=(2, 2)))(x)
x = (Dropout(0.25))(x)

x = (Conv2D(64, (3, 3), activation='relu'))(inputs)
x = (MaxPooling2D(pool_size=(2, 2)))(x)
x = (Dropout(0.25))(x)

x = (Flatten())(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.15)(x)
x_final = Dense(17, activation='sigmoid')(x)

model = Model(input=inputs, output=x_final)

sgd = SGD()
adam = Adam()
rms = RMSprop()

#lossfunct = 'mean_squared_error'
#lossfunct = 'binary_crossentropy'
lossfunct = 'categorical_crossentropy'

model.compile(optimizer=rms, loss=lossfunct,  metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=100,
          verbose=2,
          validation_data=(x_valid, y_valid))



p_valid = model.predict(x_valid, batch_size=10)
print(y_valid)
print(p_valid)
print(fbeta_score(y_valid, np.array(p_valid) > 0.5, beta=2, average='macro'))